In [1]:
OLD_PUBLIC_FIM_PERCENT = 10
NEW_PUBLIC_FIM_PERCENT = 30
NOAA_GEOPLATFORM_SUBLAYER_INDEX = 9

In [ ]:
import requests

# place full requests.get URL below
r = requests.get('https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/Updated_FIM_Implementation/FeatureServer'
                 f'/{NOAA_GEOPLATFORM_SUBLAYER_INDEX}/query?.....'
geojson = r.json()

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
features = [f for f in geojson['features']]
geom = [shape(f['geometry']) for f in features]
df = gpd.GeoDataFrame({'geom': geom})
df = df.set_geometry('geom')
df = df.set_crs('EPSG:3857')

In [ ]:
df.to_postgis(f'public_fim{NEW_PUBLIC_FIM_PERCENT}_new_only_domain', get_db_engine('viz'), schema='derived', if_exists='append')

In [3]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))
from helper_functions.shared_functions import execute_sql, sql_to_dataframe

In [36]:
sql = f'''
DROP TABLE IF EXISTS derived.public_fim{NEW_PUBLIC_FIM_PERCENT}_domain;

WITH unioned AS (
    SELECT geom FROM derived.public_fim{NEW_PUBLIC_FIM_PERCENT}_new_only_domain
    UNION
    SELECT geom FROM derived.public_fim{OLD_PUBLIC_FIM_PERCENT}_domain
)

SELECT
1 as oid,
ST_Buffer(ST_Union(ST_Buffer(geom, 1)), -1) as geom
INTO derived.public_fim{NEW_PUBLIC_FIM_PERCENT}_domain
FROM unioned
GROUP BY oid;
'''
execute_sql(sql)

In [37]:
sql_to_dataframe(f"SELECT ST_NPoints(geom) FROM derived.public_fim{NEW_PUBLIC_FIM_PERCENT}_domain;")

,st_npoints
0,1371585


In [38]:
sql = f'''
DROP TABLE IF EXISTS derived.public_fim_domain_ondeck;
SELECT *
INTO derived.public_fim_domain_ondeck
FROM derived.public_fim{NEW_PUBLIC_FIM_PERCENT}_domain;

DROP TABLE IF EXISTS derived.public_fim_domain_retired;
ALTER TABLE derived.public_fim_domain RENAME TO public_fim_domain_retired;
ALTER TABLE derived.public_fim_domain_ondeck RENAME TO public_fim_domain;
'''
execute_sql(sql)

In [39]:
from helper_functions.shared_functions import execute_sql, move_data_from_viz_to_egis, sql_to_dataframe

In [40]:
move_data_from_viz_to_egis('derived.public_fim_domain', 'reference.public_fim_domain_ondeck')

Successfully copied derived.public_fim_domain from the VIZ DB to reference.public_fim_domain_ondeck in the EGIS DB!


In [41]:
execute_sql('''
    DROP TABLE IF EXISTS reference.public_fim_domain_retired;
    ALTER TABLE reference.public_fim_domain RENAME TO public_fim_domain_retired;
    ALTER TABLE reference.public_fim_domain_ondeck RENAME TO public_fim_domain;
''', 'egis')